# Preprocessing the MRI

Preprocessing the original images follow the pipeline described in **paper**. These steps allow to aligne (using an affine transformation) the T1-weighted MR images to the MNI space. It uses mainly the [ANTs](http://stnava.github.io/ANTs/) software package. The operation to be performed are, in short: 

* **Bias field correction** using N4ITK [Tustison et al., 2010].

* **Affine registration** to the MNI152NLin2009cSym template [Fonov et al., 2011, 2009] in MNI space with the SyN algorithm [Avants et al., 2008].

* **Cropping** the registered images to remove the background.


In [2]:
# Code to perform preprocessing using clinicadl

Or you can also use the **T1_linear** pipeline of Clinica:

In [ ]:
# Command line instructions 